# [A2] Data Preparation and Regression
In this notebook the learned foundations of the **Pandas** library are applied to a **practical example**. 

The aim of this assignment is to first **examine** a dataset using an **Exploratory Data Analysis**. We want to get to know the data and analyze it using an exploratory approach. As part of this process, several methods for **dataset preparation** are introduced.

Second, the preprocessed dataset will be used to train **regression models** that **predict the monthly cold rent** based on various features such as size and condition of the apartment.

## 1: About the Dataset
A user of the data science community Kaggle developed and ran a web-scraper for the biggest **real estate platform** in Germany, **ImmoScout24**.

At three points in time (22th Sept 2018, 10th Mai 2019 and 8th Oct. 2019), all available offers at that time were retrieved from the website and stored. 
In total, data samples of over 200.000 rental properties have been collected.

The data samples contain the most important characteristics of a rental property, such as the size of the living space, the rent, both cold rent and total rent (if applicable), the location (street and house number, if available, zip code and state), energy type, etc. 
There are also two variables that contain longer free text descriptions: Description with text describing what is offered, and Amenities with a description of all available amenities, recent renovation, etc. The date column was added to indicate the time of scraping.

The dataset _immo_data.csv_ can be imported from the 'data' directory or downloaded [from kaggle](https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany) (account required). An additional file _immo_data_column_description.csv_ contains explanation for each column name.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV, train_test_split

pd.options.display.max_columns = 50

immo = pd.read_csv("data/immo_data.csv.zip")
column_descriptions = pd.read_csv("data/immo_data_column_description.csv")

immo.head(3)

As described in the first notebook, some info about the DataFrame and statistical properties of its columns can be automatically generated:

In [ ]:
display(immo.info(), immo.describe())

## 2: First Impressions
At first glance, just by looking at the list of columns, some columns seem to be duplicate.

Besides, we can already identify some columns that are expected to not be helpful for later classification problems. In practice however, you would need to verify this first!

The following code removes these columns. 

In [ ]:
immo_2 = immo.drop(["scoutId", "houseNumber", "geo_bln", "geo_krs", "geo_plz", "date"], axis=1)
immo_2.head(3)

### 2.1: ✏️ Characteristics of the Dataset
1.  How large is the (column-reduced) dataset? How many features (columns) or objects (data rows) are available?

In [ ]:
# TODO: Implement
print(f'num_cols {len(immo_2.columns)}')
print(f'num_rows {len(immo_2.index)}')

2. Output the first ten objects. Ask yourself: How do they look like? What do you notice? Do you understand the meaning of the features?

In [ ]:
# TODO: Implement
immo_2.head(10)

3. What are the data types of each feature?

In [ ]:
# TODO: Implement
print(immo_2.dtypes)

4. What are the respective maximum, average and minimum values of the following features?
- telekomUploadSpeed
- livingSpace
- heatingCosts

In [ ]:
# TODO: Implement
print(f'telekomUploadSpeed ∈ [{immo_2['telekomUploadSpeed'].min()}, {immo_2['telekomUploadSpeed'].max()}], x̄ = {immo_2["telekomUploadSpeed"].mean()}')
print(f'livingSpace ∈ [{immo_2['livingSpace'].min()}, {immo_2['livingSpace'].max()}], x̄ = {immo_2["livingSpace"].mean()}')
print(f'heatingCosts ∈ [{immo_2['heatingCosts'].min()}, {immo_2['heatingCosts'].max()}], x̄ = {immo_2["heatingCosts"].mean()}')


We split the analyzing process into two parts, each regarding either columns of numeric or non-numeric data type. 

## 3: Analyzing Categorical Data
We start with the non-numeric data columns by taking a look at their statistical characteristics: 

In [ ]:
immo_2.describe(exclude=np.number).T

The _count_ column shows the number of values in our dataset. If it is lower than our number of data samples, _NaN_ (null) values occur.

The _unique_ columns counts the number of distinct values. 

Those values that are represented the most for each column are mentioned as _top_ in the description table. The _freq_ is this most common value’s frequency.

### 3.1: One Hot Encoding

When categorical features are transformed using One Hot Encoding, a new column is created for each distinct value, as shown in the following example.

This is required for using categorical features in a numeric regression model.

#### 3.1.2: Exemplary One Hot Encoding

In the following cell the column _C2_ of the DataFrame is one-hot encoded, generating new columns that replace the original column _C2_.

In [ ]:
example_df = pd.DataFrame({"C1": ["a", "b", "a", "e"], "C2": ["b", "a", "c", "a"]})
example_df_oh = pd.get_dummies(example_df, columns=["C2"])
display(example_df, example_df_oh)

#### 3.1.2: ✏️ Removing Inappropriate Columns for One Hot Encoding
If we would want to one-hot encode the free text feature _description_, we would get a DataFrame with over 200,000 columns - so we exclude attributes with too many values for the time being.

1. In the ``immo_2`` DataFrame, identify the features that could cause too many columns when one-hot encoded and create a DataFrame ``immo_3`` that does not contain these columns

In [ ]:
# TODO: Implement
immo_3 = immo_2.drop(columns=['street','streetPlain','regio2','regio3','description','facilities'])
immo_3

2. In the "first impressions" chapter, the zip code was removed. Would you think it has value for predicting the cold rent? What would you need to consider?

In [ ]:
# TODO: answer as comment
# the zip code might be useful to have a more clear definition of where the house is located as bigger cities tend to have higher renting prices. As Zip-Codes are not numerical values it would need to be encoded before it can be used as input. Even though zip codes are numbers they have no

3. Take a look at the attribute _firingTypes_. According to the dataset author, it describes the main energy source. In the ``.describe()`` table we saw it has 133 unique values. The similar attribute _heatingType_ has only 13 unique values. Output both lists of unique values to compare the two attributes.

In [ ]:
# TODO: Implement
print(immo_2['heatingType'])
print(immo_2['firingTypes'])

4. We decide to keep the less detailed attribute and keep the dataset simple. What other option do we have? 

In [ ]:
# TODO: answer as comment
# instead of one hot encoding we could think about using embedding. We embed the heating feature using firingTypes and heatingType

5. Create a new DataFrame ``immo_4`` without the  _firingTypes_ column.

In [ ]:
# TODO: Implement
immo_4 = immo_3.drop(['firingTypes'], axis=1)

### 3.2: Visual Evaluation of Categorical Features
In this section we will take a look at the frequency distribution histograms of all remaining categorical features.

In [ ]:
categorical_columns = immo_4.select_dtypes(exclude=np.number).columns

n_cols = 5
n_rows = int(np.ceil(len(categorical_columns) / n_cols))
ax = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=False)[1].flatten()

for i, colname in enumerate(categorical_columns):
    immo_4[colname].value_counts().plot(kind="barh", ax=ax[i], title=colname, figsize=(24, 12))

plt.show()

One can see that almost all features now contain only a few unique and meaningful values.

Only _regio2_ seems to be too crowded, but that is okay as there still are some values with a lot of occurances.

The one-hot encoding itself will be done in a later chapter to keep the number of columns minimal for now.

## 4: Analyzing Numerical Data
In this section we will take a look at the numeric features. More precisely, we look mainly at outlier detection and handling.

### 4.1: Outliers
An outlier is an observation that lies an abnormal distance from other values in a given set of data. Outliers can negatively impact the quality of a regression model, which is why it is important to identify them and find a solution to deal with them.

#### 4.1.1: ✏️ Outlier Detection
Outliers can be spotted visually, e.g. when looking at a histogram or statistically by reviewing the characteristics of each column.
First, we will use the ``.describe()`` method again.

In [ ]:
# reduce the number of decimals shown in the output (for better readability)
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

immo_4.describe(include=np.number).drop("count").T

1. Take a look at the table to see if you can find some of the extreme outliers in the data by yourself.


In [ ]:
# TODO: Answer
# totalRent, heatingCosts, baseRent, serviceCharge, yearConstructed, noParkSpaces, numberOfFloors, floor, lastRefurbished


We save the names of all these suspicious columns in a list to further inspect them.
Furthermore, we see that the feature _telekomHybridUploadSpeed_ always has the value 10 or is empty, so we delete it.

In [ ]:
interesting_columns = [
    "serviceCharge",
    "totalRent",
    "yearConstructed",
    "noParkSpaces",
    "baseRent",
    "livingSpace",
    "noRooms",
    "numberOfFloors",
    "heatingCosts",
    "lastRefurbish",
]

immo_5 = immo_4.drop(["telekomHybridUploadSpeed"], axis=1)

#### 4.1.2: ✏️ Outlier Visualization
Outliers can be visualized using histograms and boxplot charts.

1. Adapt the visualization code used for categorical data to show histograms of *all* the numerical columns (instead of using ``value_counts()``)

In [ ]:
# TODO: Implement
numeric_columns = immo_5.select_dtypes(include=np.number).columns

n_cols = 5
n_rows = int(np.ceil(len(numeric_columns) / n_cols))
ax = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=False)[1].flatten()

for i, colname in enumerate(numeric_columns):
    immo_5[colname].plot(kind="hist", ax=ax[i], title=colname, figsize=(24, 12), bins=30)

plt.show()


2. Adapt the visualization code to show box plots of *only the interesting* numerical columns 

In [ ]:
# TODO: Implement
numeric_columns = immo_5.select_dtypes(include=np.number).columns

n_cols = 5
n_rows = int(np.ceil(len(numeric_columns) / n_cols))
ax = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=False)[1].flatten()

for i, colname in enumerate(numeric_columns):
    immo_5[colname].plot(kind="box", ax=ax[i], title=colname, figsize=(24, 12))

plt.show()

#### 4.1.3: Outlier Removal
Looking at the plots, we decide that these are too many outliers to adress them individually in this excercise.

Instead, we take look at what would happen if we would simply cut off the top and bottom 0.5% of the data.

In [ ]:
upper_limits  = immo_5[interesting_columns].quantile(0.005)
lower_limits = immo_5[interesting_columns].quantile(0.995)

limits = pd.DataFrame({"lower bound": upper_limits, "upper bound": lower_limits})
limits


This looks like a healthier distribution. 
We therefore create a new DataFrame containing only the rows that fit within these bounds (plus those that are _NaN_, these we will care about later):

In [ ]:
immo_6 = immo_5.copy()

for colname, (lower, upper) in limits.iterrows():
    col = immo_5[colname]
    # immo_6[colname].loc[:] = np.nan
    immo_6[colname] = immo_5[((col <= upper) & (col >= lower)) | col.isna()][colname]

We can compare the statistical characteristics and number of _NaN_ cells before and after the conversion to check if everything worked as intended:

In [ ]:
nans_5 = immo_5.select_dtypes(include=np.number).isna().sum().rename("count_NaN")
nans_6 = immo_6.select_dtypes(include=np.number).isna().sum().rename("count_NaN")
desc_5 = immo_5.describe(include=np.number).drop("count").T.join(nans_5)
desc_6 = immo_6.describe(include=np.number).drop("count").T.join(nans_6)

pd.concat([desc_5, desc_6, desc_6 - desc_5], axis=1, keys=["immo_5", "immo_6", "Δ"])

We see that the minimum and maximum values look far more reasonable now. However, the number of _NaN_ fields has increased, as our conversion simply replaced all outlier values with _NaN_.

If this is a valid approach depends very much on your task and often times it would make more sense to instead drop all outlier rows. 

At the very latest, it is now time to take care of the NaN values.

## 5: Handling Missing Values
Let's take a look at how many missing values we have for both kind of data types:


In [ ]:
nans = immo_6.isna().sum().rename("count_NaN").sort_values(ascending=False)
dtypes = immo_6.dtypes.rename("dtype")
display(nans.to_frame().join(dtypes))
print("Total Number of NaNs:", immo_6.isna().sum().sum())

Some of these columns contain many _NaN_ values, if we ignored all these, there would not be many rows left. 

We therefore follow this strategy first: 

- For numeric values, the mean of the remaining values is used for a missing value.

- For non-numeric values, the most frequent value (=mode) of the remaining values is used for a missing value.

This is a very simple approach, but it obviously will alter the true distribution of our data.

### 5.1 ✏️ Replacing Missing Values
To fill _NaN_ values, pandas provides the ``.fillna()`` method. It simply replaces all _NaNs_ with a certain value.

The machine learning library **scikit-learn** provides a more advanced interface, the [_SimpleImputer_](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html).
In contrast, it can replace missing values using a descriptive statistic like mean, median, or the most frequent value along each column.

1. Create a _SimpleImputer_ object to replace all _NaN_ values in numerical columns with the mean of the respective column. Apply it to _immo_7_ and replace the respective columns with the new data. Finally, print the total number of NaNs in the new dataframe.

In [ ]:
immo_7 = immo_6.copy()

numeric_columns = immo_7.select_dtypes(include=np.number).columns

# TODO: Implement
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
immo_7.loc[:,numeric_columns] = imp_mean.fit_transform(immo_7[numeric_columns])
print("Total Number of NaNs:", immo_7.isna().sum().sum())

2. Create a new DataFrame _immo_8_ in which _NaN_ values in all non-numeric columns are replaced with the most frequent value of that column. Again, print the number of NaNs in the new DataFramne (should be 0).  

In [ ]:
immo_8 = immo_7.copy()

imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_columns = immo_8.select_dtypes(exclude=np.number).columns
immo_8.loc[:,categorical_columns] = imp_freq.fit_transform(immo_8[categorical_columns])

print("Total Number of NaNs:", immo_8.isna().sum().sum())

## 6: Feature Engineering
In general, feature engineering is the process of transforming raw data to meaningful features that can be used for machine learning.

So far, we have cleaned the raw data to make it easier to decide which features to create.

Typical engineered features include[¹](https://en.wikipedia.org/wiki/Feature_engineering):

- for numerical data
    - Numerical transformations (like taking fractions or scaling)
    - Clustering
    - Group aggregated values
    - Principal component analysis
    - Constructed metrics based on proven scientific research in the specific domain.
- for categorical data
    - Encoded Category encoder (like one-hot or target encoding)

### 6.1: Analyzing Correlations
We generate scatter plots to identify obvious linear or nonlinear relationships between individual features and the target variable *baseRent*.

In [ ]:
numeric_columns = list(immo_8.select_dtypes(include=np.number).columns)
numeric_columns.remove("baseRent")

n_cols = 5
n_rows = int(np.ceil(len(numeric_columns) / n_cols))
ax = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=False)[1].flatten()

for i, colname in enumerate(numeric_columns):
    immo_8[["baseRent",colname]].plot.scatter(x=colname, y="baseRent", ax=ax[i], figsize=(24, 14), alpha=0.2)

plt.show()

✏️ Take a moment to understand the graph. Which correlations do you notice?

➡️ The data scatters relatively broadly. As expected, there is correlation between size and rental price.
Of course, there also is a correlation between the target feature and closely related features like _baseRent_ - we will handle this in the next chapter. 
For now, we see no further anomalies.

We can also plot the correlation coefficient for all combinations of features (= a correlation matrix) to see which features are similar to each other.
For this, we use the **seaborn** library that acts as a wrapper for commonly used matplotlib plots.

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(immo_8.corr(numeric_only=True))
plt.show()

Other than the already noted correlations to _baseRent_, we only see new correlations in between features and their corresponding _range_ features (e.g. _noRooms_ and _noRoomsRange_). That is acceptable and we will continue without further action.

In this example, we will not create any new features other than the one hot encoding.

### 6.2 ✏️ Applying the One Hot Encoding
Next, we transform the categorical columns using OneHot encoding. 

1. Apply the code mentioned in the One Hot Encoding example from earlier to encode the categorical columns of our DataFrame.

In [ ]:
immo_9 = immo_8.copy()

# TODO: Implement
categorical_columns = immo_9.select_dtypes(exclude=np.number).columns
numerical_columns = immo_9.select_dtypes(include=np.number).columns
one_hot = pd.get_dummies(immo_9[categorical_columns])
immo_9 = one_hot.merge(immo_9[numerical_columns], left_index=True, right_index=True)
immo_9.head(1)

Let's take a look at the created DataFrame:

In [ ]:
print("Shape:", immo_9.shape, ", Columns:", list(immo_9.columns))
immo_9.head(2)

Altogether, we now have a data matrix with 268850 records (rows) and 515 features (columns).

## 7: Handling the Target Column
Finally, we will need to extract the target column and remove all columns that might lead the algorithm to overfit because of the high correlation.

Keeping these "easy" columns in the dataset would almost be cheating ;)

Furthermore, it makes sense to remove all rows that do not contain a value for the target column.

In [ ]:
immo_X = immo_9[immo_9["baseRent"].isna() == False]

immo_Y = immo_X["baseRent"]
immo_X = immo_X.drop(["baseRent", "totalRent", "baseRentRange"], axis=1)
print(immo_Y.shape, immo_X.shape)

In a real world example, you might want to export your final DataFrame now, e.g. to a new csv file.

````python
immo_X.to_csv('immoscout_prepared_X.csv', index=False)
````

To keep state consistent between students, we will however not build upon this file now but instead rely on the methods introduced in the following chapter.

## 8: Summary of Data Preparation
In production, the steps to import and explore the dataset can be summarized in a small number of functions. 

The goal of this section is to once again read the dataset file and perform only the core preparation steps we identified so far:
- delete irrelevant and target-related columns
- delete outliers
- impute NaN values
- one-hot encode categorical features
- remove rows without label
- extract target column

We define re-usable methods (for this specific dataset) and then call them subsequently.

In [ ]:
def drop_columns(df):
    return df.drop(
        [
            "scoutId",
            "houseNumber",
            "geo_bln",
            "geo_krs",
            "geo_plz",
            "date",
            "street",
            "streetPlain",
            "description",
            "facilities",
            "regio3",
            "firingTypes",
            "telekomHybridUploadSpeed",
            "totalRent",
            "baseRentRange",
        ],
        axis=1,
    )


def remove_outliers(df):
    dfc = df.copy()
    columns_with_outliers = [
        "serviceCharge",
        "yearConstructed",
        "noParkSpaces",
        "baseRent",
        "livingSpace",
        "noRooms",
        "numberOfFloors",
        "heatingCosts",
        "lastRefurbish",
    ]

    upper_limits = dfc[columns_with_outliers].quantile(0.995)
    lower_limits = dfc[columns_with_outliers].quantile(0.005)
    for colname in columns_with_outliers:
        col = dfc[colname]
        dfc = dfc[
            ((col <= upper_limits[colname]) & (col >= lower_limits[colname]))
            | col.isna()
        ]
    return dfc


def remove_rows_with_NaN_target(df):
    return df[df["baseRent"].isna() == False]


def impute_NaNs(df):
    dfc = df.copy()
    categorical_columns = dfc.select_dtypes(exclude=np.number).columns
    imp_freq = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
    dfc.loc[:, categorical_columns] = imp_freq.fit_transform(dfc[categorical_columns])

    numeric_columns = dfc.select_dtypes(include=np.number).columns
    imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
    dfc.loc[:, numeric_columns] = imp_mean.fit_transform(dfc[numeric_columns])
    return dfc


Calling these methods now to prepare data based on the raw original dataset is easy and produces very well readable code:

In [ ]:
immo_p = drop_columns(immo)
immo_p = remove_outliers(immo_p)
immo_p = remove_rows_with_NaN_target(immo_p)
immo_p = impute_NaNs(immo_p)
immo_p = pd.get_dummies(immo_p)
immo_p_Y = immo_p.pop("baseRent")

immo_p.head()


## 9: Splitting and Scaling
In addition to the steps we learned in the last notebook, certain methods can be useful depending on your chosen model architecture and error metric.

### 9.1: Split Training and Test Dataset
Splitting the data is important for every supervised learning task. Seperating training and test data ensures that the model is not tested against data it has already seen during the training phase.

In the following example, the ``train_test_split()`` method from scikit-learn is used to create a test-split of 20% the size of the full dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    immo_p, immo_p_Y, test_size=0.2, random_state=42
)

Most of scikit-learn is Pandas-compatible and will accept and return DataFrames if feasible. The same method can be used to split numpy arrays, in which case it would return a numpy array instead.

### 9.2: Scaling
Feature scaling, sometimes called normalization, is a technique used to standardize the independent features of a dataset to a fixed range. 

If values or units are of different magnitudes, some ML methods tend to overfit on high values while disregarding smaller values.

Scikit-Learn provides multiple [methods for scaling data](https://scikit-learn.org/stable/modules/preprocessing.html).

In this example, we will use the ``MinMaxScaler`` to scale each feature individually to the range $[0, 1]$.

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 10: Linear Regression
As introduced in the lecture, a linear regression tries to fit a linear function of the form $ f(X) = w_0 + \sum^{|X|}_{j=1} (w_j * x_j) $ by learning weights $W$ that minimize the error $d$ between $f(X)$ and the actual label $y$ for a given feature-vector $X$. In our case, $X$ represents a row (data sample) of the DataFrame, and $x_j$ therefore is a certain features value for this row.

The error $d$ is measured using a loss function, often times the least squares approach is used with $ d(X_i, y_i) = (y_i - f(X_i) )^2 $, with the $i$ indicating that we can calculate the distance for different rows (feature-vectors $X_i$), each having it's own label $y_i$. 

In Training, we calculate the total loss by iterating over all rows: $ L(W) = \frac{\sum^N_{i=1}(d_i)}{N}$. 

Using scikit-learn, this math is hidden from the programmer. However, it is stil important to understand these basics to differentiate and choose the right architecture for a given problem.

### 10.1: Train
During training, the weights are learned and the function is _fitted_. Hence the name of the scikit-learn function:

In [ ]:
model_lr = linear_model.LinearRegression()
model_lr.fit(X_train_scaled, y_train)

The model has now learned several weights (coefficients in the function) that can be accessed using a attribute:

In [ ]:
model_lr.coef_[0:10]

### 10.2: Test
We can now apply the fitted function $f(X)$ for various $X$. In this case, we test both the test and the training set, even though only the test involving the test data has significance for the real-world accuracy. 

In [ ]:
y_test_pred = model_lr.predict(X_test_scaled)
y_train_pred = model_lr.predict(X_train_scaled)

print(y_test_pred[0:5])

### 10.3: Alternative Approach: Scikit-Learn Pipelines
A common mistake is to apply scaling methods to the entire data instead of train and test splits individually, leaking information from the test set to the training set.

Additionally, it is easy to forget which part of the data has to be scaled for which steps of the machine learning process.
  
It therefore is recommended to call the above methods within a [Pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline) in order to prevent most risks of data leaking: 


In [ ]:
pipe_lr = make_pipeline(MinMaxScaler(), linear_model.LinearRegression())
pipe_lr.fit(X_train, y_train)

y_test_pred_pipe = pipe_lr.predict(X_test)
y_train_pred_pipe = pipe_lr.predict(X_train)

print(y_test_pred_pipe[0:5])

All scaling will automatically be done if calling the methods of the pipeline.

### 10.4: Evaluation

We will evaluate the trained models using the following metrics:
- R²
- Mean Squared Error (MSE) 
- Mean Absolute Error (MAE)

They are all available in the sklearn framework and will be applied for the train and test splits separately.

In [ ]:
def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names):
    train = pipeline_or_model.score(X_train, y_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    r2_test = pipeline_or_model.score(X_test, y_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'R²':>10} | {'MSE':>14} | {'MAE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {train:10.2f} | {mse_train:14.2f} | {mae_train:10.2f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {r2_test:10.2f} | {mse_test:14.2f} | {mae_test:10.2f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )


print_evaluation(model_lr, X_train_scaled, X_test_scaled, y_train, y_test, y_train_pred, y_test_pred, immo_p.columns)
print_evaluation(pipe_lr, X_train, X_test, y_train, y_test, y_train_pred_pipe, y_test_pred_pipe, immo_p.columns)


As you can see, using the pipeline produces equal results.

### 10.5: Visualization
The result of this 512 dimensional multiple linear regression is not easy to visualize in 2D or 3D graphs.

To still get some visual insight to our accuracy, we create a scatterplot showing samples of the test set in two dimensions: 
The size of the apartment (X-axis) and the cold rent (Y-axis), both as groundtruth and as prediction. 

For further insight, feel free to create more scatterplots.

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(X_test["livingSpace"], y_test, "bo", alpha=0.1, label="ground truth")
plt.plot(X_test["livingSpace"], y_test_pred, "ro", alpha=0.1, label="predicted")
plt.gca().update(dict(title="Immo Prediction", xlabel="appartment size", ylabel="cold rent"))
plt.legend(title="cold rent")
plt.show()

We can see that they roughly overlap, however the points are far from matching perfectly.

## 11: ✏️ Linear Regression Excercises
1. Copy and adapt the code from above: How does the prediction quality change when outliers are not removed?

In [ ]:
immo_p2 = drop_columns(immo)
immo_p2 = remove_rows_with_NaN_target(immo_p2)
immo_p2 = impute_NaNs(immo_p2)
immo_p2 = pd.get_dummies(immo_p2)
immo_p2_Y = immo_p2.pop("baseRent")

# splitting data
X2_train, X2_test, y2_train, y2_test = train_test_split(
    immo_p2, immo_p2_Y, test_size=0.2, random_state=42
)

model_lr = linear_model.LinearRegression()
model_lr.fit(X2_train, y2_train)

y2_test_pred = model_lr.predict(X2_test)
y2_train_pred = model_lr.predict(X2_train)

# def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names):
print("Unscaled result:")
print_evaluation(model_lr, X2_train, X2_test, y2_train, y2_test,y2_train_pred, y2_test_pred, immo_p2.columns)
print("Scaled result:")
print_evaluation(pipe_lr, X_train, X_test, y_train, y_test, y_train_pred_pipe, y_test_pred_pipe, immo_p.columns)

2. Create a region-specific model, trained on only a certain geographical region (try it once with Baden-Württemberg and another time with just Karlsruhe)
   Which model does show an overfitting problem: the model of Baden-Württemberg or the model of Karlsruhe? 

In [ ]:
# Baden-Württemberg dataset immo_p3 and the target is immo_p3_Y
immo_p3 = drop_columns(immo)
# TODO: Implement
immo_p3 = immo_p3[immo_p3['regio1'].str.contains("Baden_Württemberg")]
immo_p3 = remove_rows_with_NaN_target(immo_p3)
immo_p3 = remove_outliers(immo_p3)
immo_p3 = impute_NaNs(immo_p3)
immo_p3 = pd.get_dummies(immo_p3)
immo_p3_Y = immo_p3.pop("baseRent")

immo_p3.head()

In [ ]:

# data splitting
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(
    immo_p3, immo_p3_Y, test_size=0.2, random_state=42
)

# Model and its evaluation
## TODO: Implement
pipe_lr3 = make_pipeline(MinMaxScaler(), linear_model.LinearRegression())
pipe_lr3.fit(X_train_3, y_train_3)

y_test_pred_pipe_3 = pipe_lr3.predict(X_test_3)
y_train_pred_pipe_3 = pipe_lr3.predict(X_train_3)

# def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names):
print_evaluation(pipe_lr3, X_train_3, X_test_3, y_train_3, y_test_3, y_train_pred_pipe_3, y_test_pred_pipe_3, immo_p3.columns)

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(X_test_3["livingSpace"], y_test_3, "bo", alpha=0.3, label="ground truth")
plt.plot(X_test_3["livingSpace"], y_test_pred_pipe_3, "ro", alpha=0.3, label="predicted")
plt.gca().update(dict(title="Immo Prediction", xlabel="appartment size", ylabel="cold rent"))
plt.legend(title="cold rent")
plt.show()

In [ ]:
# Karlsruhe dataset immp_p4 and the target is immo_p4_Y
immo_karlsruhe = immo[immo['geo_krs'].str.contains("Karlsruhe")]
immo_p4 = drop_columns(immo_karlsruhe)
# TODO: Implement
immo_p4 = remove_rows_with_NaN_target(immo_p4)
immo_p4 = remove_outliers(immo_p4)
immo_p4 = impute_NaNs(immo_p4)
immo_p4 = pd.get_dummies(immo_p4)
immo_p4_Y = immo_p4.pop("baseRent")
immo_karlsruhe.head()

In [ ]:

# data splitting
X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(
    immo_p4, immo_p4_Y, test_size=0.2, random_state=42
)

# Model and its evaluation
## TODO: Implement
pipe_lr4 = make_pipeline(MinMaxScaler(), linear_model.LinearRegression())
pipe_lr4.fit(X_train_4, y_train_4)

y_test_pred_pipe_4 = pipe_lr4.predict(X_test_4)
y_train_pred_pipe_4 = pipe_lr4.predict(X_train_4)

print_evaluation(pipe_lr4, X_train_4, X_test_4, y_train_4, y_test_4,y_train_pred_pipe_4, y_test_pred_pipe_4, immo_p4.columns)

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(X_test_4["livingSpace"], y_test_4, "bo", alpha=0.3, label="ground truth")
plt.plot(X_test_4["livingSpace"], y_test_pred_pipe_4, "ro", alpha=0.3, label="predicted")
plt.gca().update(dict(title="Immo Prediction", xlabel="appartment size", ylabel="cold rent"))
plt.legend(title="cold rent")
plt.show()

3. Predict the cold rent for your own appartment. Think about how the feature vector would look like for your appartment, create a DataFrame with one row and apply the best performing of the previously learned models using ``.predict()``. Take a look at ``column_descriptions`` if you are unsure of what a certain column means. 💡 This excercise is not as easy as it sounds, as you need to perform the preprocessing steps while making sure NaN values are correctly filled and one-hot-encoding works as intended to get exactly the same DataFrame structure used during fitting the model!


In [ ]:
my_apartment_data = {
    'regio1': 'Baden_Württemberg',
    'serviceCharge': 20,
    'newlyConst': False,
    'balcony': False,
    'telekomUploadSpeed': 1000.0,
    'noParkSpaces': 0,
    'hasKitchen': True,
    'cellar': False,
    'heatingType': 'gas_heating',
    'yearConstructedRange': 1,
    'livingSpace': 12,
    'condition': 'well_kept',
    'interiorQual': 'normal',
    'lift': False,
    'noRooms': 1,
    'floor': 2,
    'numberOfFloors': 4,
    'garden': False,
    'noRoomsRange': 1,
    'livingSpaceRange': 1,
    'telekomTvOffer': 'NONE',
    'regio2': 'Karlsruhe',
    'typeOfFlat': 'apartment',
    'description': 'EE appartment with a nice view',
}

df = pd.DataFrame([my_apartment_data])

In [ ]:
my_apartment_data = pd.concat([immo, df], ignore_index=True)

my_apartment_data = drop_columns(my_apartment_data)
immo_temp = drop_columns(immo)
my_apartment_data = impute_NaNs(my_apartment_data)

categorical_columns = immo_temp.select_dtypes(exclude=np.number).columns
numerical_columns = immo_temp.select_dtypes(include=np.number).columns
one_hot = pd.get_dummies(my_apartment_data[categorical_columns])
my_apartment_data = my_apartment_data[numerical_columns].merge(one_hot, left_index=True, right_index=True)

my_apartment_data = pd.get_dummies(my_apartment_data)
my_apartment_data = my_apartment_data.reindex(immo_p.columns, axis=1)

In [ ]:
my_apartment_data_X = my_apartment_data.tail(1)

pipe_lr.predict(my_apartment_data_X)

## 12: Regularization 

**Regularization** in ML is a term that describes methods to simplify a regression problem and **reduce the generalization error** (error on test data, not training data). It can be separated into two kinds:

- In **Explicit regularization** one explicitly adds a bias (penalty/regularization) term to the optimization problem.
    - Ridge Regression (L2 regularization): the cost function is altered by adding the _Ridge Regression penalty_ bias term (squared magnitude of the coefficient) to it.
    - Lasso Regression (L1 regularization): "Least Absolute and Selection Operator": similar to the Ridge Regression except that the penalty term contains only the **absolute** weights instead of a square of weights
        - Since it takes absolute values, hence, it can shrink the slope to 0, whereas Ridge Regression can only shrink it near 0.
- **Implicit regularization** is all other forms of regularization. This includes, for example, early stopping, using a robust loss function, and discarding outliers. Implicit regularization is essentially ubiquitous in modern machine learning approaches, including stochastic gradient descent for training deep neural networks, and ensemble methods (such as random forests and gradient boosted trees).

In this chapter we utilize the L1 & L2 regularization.

### 12.1: ✏️ Regression with Regularization
Using the L1 and L2 regularization is simple when using Scikit-Learn. 

1. Read [the documentation](https://scikit-learn.org/stable/modules/linear_model.html) to find the right methods for L1 and L2 regularization. Train both models on the geographical region from task 11.2 that showed an overfitting problem (Baden-Württemberg "immo_p3" or Karlsruhe "immo_p4") and print the evaluation. 

In [ ]:
# TODO: Implement
from sklearn.linear_model import Ridge
# Ridge is L2 regularization
# will take in its fit method arrays X, y and will store the w coefficients of the linear model in its coef_ member
model_3 = Ridge()
pipe_lr3 = make_pipeline(MinMaxScaler(), model_3)
pipe_lr3.fit(X_train_3, y_train_3)

y_test_pred_pipe_3 = pipe_lr3.predict(X_test_3)
y_train_pred_pipe_3 = pipe_lr3.predict(X_train_3)

print_evaluation(pipe_lr3, X_train_3, X_test_3, y_train_3, y_test_3, y_train_pred_pipe_3, y_test_pred_pipe_3, immo_p3.columns)

In [ ]:
model_4 = linear_model.Lasso()
# The Lasso is a linear model that estimates sparse coefficients.
# The lasso estimate thus solves the minimization of the least-squares penalty with a||w|| added, where a is a constant and ||w|| is the -norm of the coefficient vector.
pipe_lr = make_pipeline(MinMaxScaler(), model_4)
pipe_lr.fit(X_train_4, y_train_4)
y_test_pred_pipe_4 = pipe_lr.predict(X_test_4)
y_train_pred_pipe_4 = pipe_lr.predict(X_train_4)
print_evaluation(pipe_lr, X_train_4, X_test_4, y_train_4, y_test_4,y_train_pred_pipe_4, y_test_pred_pipe_4, immo_p4.columns)

2. Both methods accept a parameter ``alpha``. What can this parameter be used for? Apply the same code from the previous excercise for another alpha value.

In [ ]:
# TODO: Answer and Implement
# The alpha parameter controls the degree of sparsity of the estimated coefficients.

### 12.3: Hyperparameter Optimization
It now has become apparent that the parameter ``alpha`` can influence the quality of our L1 and L2 models.
However, this is just one example. Almost every ML model can be parameterized and finding the best parameters is a challenging task.

Systematic approaches to finding ideal parameters are called [hyperparameter optimization](https://en.wikipedia.org/wiki/Hyperparameter_optimization).
They are based on the idea of systematically trying out different values for each parameters and evaluating which of them work best. 

A traditional but still popular hyperparameter optimization approach is called **grid search**, which is also provided as a [scikit-learn module](https://scikit-learn.org/stable/modules/grid_search.html).



In [ ]:
# The used dataset here is immo_p4 you can try another dataset 

grid_clf = GridSearchCV(
    linear_model.Ridge(),
    {"alpha": [1e-3, 1e-2, 1e-1, 1, 10, 100]},
    n_jobs=-1,
    return_train_score=True,
    verbose=5,
)

pipe_grid = make_pipeline(MinMaxScaler(), grid_clf)
pipe_grid.fit(X_train_4, y_train_4)

The results of this optimization can be accessed as an attribute of the gridsearch object. 
We quickly convert the returned dictionary to a Pandas DataFrame for easier reading.

In [ ]:
pd.DataFrame(grid_clf.cv_results_).set_index("params").sort_values("mean_test_score", ascending=False)


Further use of the pipeline object will automatically use the best estimator, which is refitted with the best found parameters on the whole training dataset.

In [ ]:
y_test_pred_grid = pipe_grid.predict(X_test_4)
y_train_pred_grid = pipe_grid.predict(X_train_4)

print_evaluation(
    pipe_grid,
    X_train_4,
    X_test_4,
    y_train_4,
    y_test_4,
    y_train_pred_grid,
    y_test_pred_grid,
    immo_p4.columns,
)


### 12.4: K-Fold Cross-Validation
If you paid close attention to the GridSearch documentation or the returned result table, you might have already seen that the hyperparameter optimizer uses a technique called **K-Fold Cross-Validation**, whose basic idea is as follows: 

1. The total dataset $T$ is randomly divided into $k$ equally sized subsets (splits/folds) $T_1 \dots T_k$.
2. For each iteration $i_1 \dots i_k$ a different subset $T_i$ is used as the test dataset and the remaining data $(T \setminus T_i)$ is used as the training dataset.
3. The mean value of the accuracies of the individual iterations is used as the overall result of the cross-validation.

This way, no explicit splitting of training and testing set is necessary, instead, a variety of splits will be automatically done and evaluated.
This has the advantage that meaningless accuracies of bad splits (e.g. when a certain class is only represented in test split but never in the training split) are averaged out and the mean_accuracy is more representative for the full dataset (and not just the current split).

In theory, we can run cross-validation on the full dataset (``immo_1``) and not just on the training split we defined earlier (``X_train`` and ``y_train``), however we would then have no more data to predict (or do a final evaluation on, independendent of all regularization techniques etc.). Depending on your task, you might also not have access to the test set labels but instead can only submit your predictions to a third party that can calculate you a score for it (Common for data-science challenges like on [Kaggle](https://www.kaggle.com/) or the [Data-Mining-Cup](https://www.data-mining-cup.com/)).

In the previous example using GridSearch, a 5-fold cross-validation has automatically been used for each value of alpha. Scikit-Learn provides integrated models as well, e.g. [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

Of course, it is also possible to manually call the helper methods for cross-valdiation, and a simple grid search can also be done manually:

In [ ]:
# The used dataset here is immo_p4 you can try another dataset 

results_kfcv = {}

# Loop all alphas we want to try
for alpha in [1e-3, 1e-2, 1e-1, 1, 10, 100]:
    scores = []
    # Loop over 5 different Train/Test-Splits
    splits = KFold(n_splits=5, shuffle=True, random_state=42).split(X_train_4)
    for train_index, test_index in splits:
        scores.append(
            make_pipeline(MinMaxScaler(), linear_model.Ridge(alpha=alpha))
            .fit(X_train.iloc[train_index], y_train.iloc[train_index])
            .score(X_train.iloc[test_index], y_train.iloc[test_index])
        )
    # Save mean and standard deviation over the k=5 runs for every alpha
    results_kfcv[alpha] = {
        # "scores": scores,
        "mean_score": np.mean(scores),
        "stddev_score": np.std(scores),
    }

pd.DataFrame(results_kfcv).T.sort_values("mean_score", ascending=False)

The best performing alpha can again be easily retreived from these results. It can then be used as a parameter for training a new scikit-learn model with the whole training dataset.

### 12.5: ✏️ Advanced Regression Exercise:
1. Evaluate which out of three alpha values that you define work best when fitting a _Lasso_ regression model on the geographical region from task 11.2 that showed an overfitting problem (Baden-Württemberg "immo_p3" or Karlsruhe "immo_p4") using a 3-Fold Cross-Validation

In [ ]:
# TODO: Implement
# The used dataset here is immo_p4 you can try another dataset

results_kfcv = {}

# Loop all alphas we want to try
for alpha in [1e-2, 1e-1, 1]:
    scores = []
    # Loop over 5 different Train/Test-Splits
    splits = KFold(n_splits=3, shuffle=True, random_state=42).split(X_train_3)
    for train_index, test_index in splits:
        scores.append(
            make_pipeline(MinMaxScaler(), linear_model.Lasso(alpha=alpha))
            .fit(X_train.iloc[train_index], y_train.iloc[train_index])
            .score(X_train.iloc[test_index], y_train.iloc[test_index])
        )
    # Save mean and standard deviation over the k=5 runs for every alpha
    results_kfcv[alpha] = {
        # "scores": scores,
        "mean_score": np.mean(scores),
        "stddev_score": np.std(scores),
    }

pd.DataFrame(results_kfcv).T.sort_values("mean_score", ascending=False)

In [ ]:
# TODO: Implement
# The used dataset here is immo_p4 you can try another dataset

results_kfcv = {}

# Loop all alphas we want to try
for alpha in [0.1, 1, 10]:
    scores = []
    # Loop over 5 different Train/Test-Splits
    splits = KFold(n_splits=3, shuffle=True, random_state=42).split(X_train_4)
    for train_index, test_index in splits:
        scores.append(
            make_pipeline(MinMaxScaler(), linear_model.Lasso(alpha=alpha))
            .fit(X_train.iloc[train_index], y_train.iloc[train_index])
            .score(X_train.iloc[test_index], y_train.iloc[test_index])
        )
    # Save mean and standard deviation over the k=5 runs for every alpha
    results_kfcv[alpha] = {
        # "scores": scores,
        "mean_score": np.mean(scores),
        "stddev_score": np.std(scores),
    }

pd.DataFrame(results_kfcv).T.sort_values("mean_score", ascending=False)